In [1]:
import pandas as pd
import plotly.express as px
import sqlite3
import numpy as np
from src.utils import get_climb_score, EncoderDecoder

In [2]:
# load everything from sql
conn = sqlite3.connect("data/db.sqlite3")
climbs = pd.read_sql_query("SELECT * FROM climbs", conn)
grades = pd.read_sql_query("SELECT * FROM difficulty_grades", conn)
stats = pd.read_sql_query("SELECT * FROM climb_stats", conn)
holds = pd.read_sql_query("SELECT * FROM holes", conn)
placements = pd.read_sql_query("SELECT * FROM placements", conn)
holds = pd.merge(placements, holds, left_on="hole_id", right_on="id")
holds.set_index("id_x", inplace=True)

In [3]:
# merge and rename
df = pd.merge(
    climbs.drop("angle", axis=1), stats, left_on="uuid", right_on="climb_uuid"
)
df["display_grade"] = df["display_difficulty"].apply(
    lambda x: grades.loc[int(x) + 1, "boulder_name"]
)
df["average_grade"] = df["difficulty_average"].apply(
    lambda x: grades.loc[int(x) + 1, "boulder_name"]
)

In [4]:
LAYOUT_ID = 1
MIN_ASCENTS = 1
MIN_QUALITY = 2

print(df.shape)
df = df[df["frames_count"] == 1]
print(df.shape)
df = df[df["is_listed"] == 1]
print(df.shape)
df = df[df["layout_id"] == 1]
print(df.shape)
df = df[df["quality_average"] >= MIN_QUALITY]
print(df.shape)
df = df[df["ascensionist_count"] >= MIN_ASCENTS].reset_index()
print(df.shape)

holds = holds[holds["layout_id"] == 1]
holds = holds[holds.index.to_series() < 3000]
encdec = EncoderDecoder(holds)

(158652, 28)
(157784, 28)
(149859, 28)
(127969, 28)
(124616, 28)
(124616, 29)


In [6]:
used_holds = set()
colors = set()
bad_route_ids = set()
for name, row in df.iterrows():
    for frame in row["frames"].split("p")[1:]:
        hold, color = frame.split("r")
        if int(color) not in [12, 13, 14, 15] or int(hold) not in holds.index:
            bad_route_ids.add(name)
            break
        used_holds.add(int(hold))
        colors.add(int(color))
print(df.shape)
df = df.loc[list(set(df.index).difference(bad_route_ids))]
print(df.shape)
df["score"] = df["frames"].apply(lambda x: get_climb_score(encdec(x, 0)))
df = df[df["score"] >= 3.5]
print(df.shape)
df.groupby("frames")
split_values = np.random.choice(
    ["train", "test", "val"], size=len(df), p=[0.7, 0.2, 0.1]
)
df["split"] = split_values
train = df[df["split"] == "train"]
test = df[df["split"] == "test"]
val = df[df["split"] == "val"]

(123368, 29)
(123368, 29)


/home/ilya/Projects/kilter/src/utils.py:34: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  x, y = (argmax // climb.size(2), argmax % climb.size(2))
/home/ilya/Projects/kilter/src/utils.py:40: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  x, y = (argmax // climb.size(2), argmax % climb.size(2))


(108163, 30)


In [ ]:
def add_split_column(df):
    split = np.random.choice(["train", "test", "val"], p=[0.7, 0.2, 0.1])
    df["split"] = split
    return df

In [7]:
!mkdir -p data/raw
df.to_csv("data/raw/all_climbs.csv")
train.to_csv("data/raw/train.csv")
test.to_csv("data/raw/test.csv")
val.to_csv("data/raw/val.csv")
holds.to_csv("data/raw/holds.csv")
grades.to_csv("data/raw/grades.csv")

In [8]:
df["frames"].to_csv("frames.txt", index=False, header=False)

In [9]:
df = df.drop_duplicates("frames")

In [10]:
df["frames"].apply(lambda x: " ".join(x.split("p")[1:])).to_csv(
    "../minimal-text-diffusion/data/frames-split.txt", index=False, header=False
)